# Discussion Module 4

Looking at network models, converting primal to duals, and sensitivity analysis stuff

In [110]:
# from pulp import LpProblem, LpMinimize, LpVariable
import pulp as plp
import numpy as np
import pandas as pd
# import lpsolve55

In [2]:
import pulp as plp
plp.listSolvers(onlyAvailable=True)

['GLPK_CMD', 'PULP_CBC_CMD']

In [91]:
# first create the problem

prob = plp.LpProblem('CarsTransfers', plp.LpMinimize)

# define the variables and constraints for each location
# since this is a network problem, the variable are going to be the transfers
# and the constraints are going to be the number each location can give or needs
 
# from Newark
N2B = plp.LpVariable("Newark_to_Boston", lowBound=0)
N2R = plp.LpVariable("Newark_to_Richmond", lowBound=0)

# from Jacksonville
J2R = plp.LpVariable("Jacksonville_to_Richmond", lowBound=0)
J2A = plp.LpVariable("Jacksonville_to_Atlanta", lowBound=0)
J2M = plp.LpVariable("Jacksonville_to_Mobile", lowBound=0)

# from Boston
B2C = plp.LpVariable("Boston_to_Columbus", lowBound=0)

# from Columbus
C2A = plp.LpVariable("Columbus_to_Atlanta", lowBound=0)

# from Atlanta
A2R = plp.LpVariable("Atlanta_to_Richmond", lowBound=0)
A2M = plp.LpVariable("Atlanta_to_Mobile", lowBound=0)
A2C = plp.LpVariable("Atlanta_to_Columbus", lowBound=0)

# From Mobile
M2A = plp.LpVariable("Mobile_to_Atlanta", lowBound=0)


# constraints -- needed or available per city
prob += N2B + N2R                          <= 200 # Newark -- only have 200 cars_to_give. Swapped it to gte to make it easier for the solver
prob += J2R + J2A + J2M                    <= 300 # Jacksonville -- only have 300 cars_to_give. Swapped it to gte to make it easier for the solver
prob += N2B - B2C                          >= 100 # Boston -- need_to_have at least 100 cars
prob += B2C + A2C - C2A                    >= 60  # Columbus -- need_to_have at least 60 cars
prob += C2A + M2A + J2A - A2C - A2M - A2R  >= 170 # Atlanta > 170 
prob += A2M + J2M - M2A                    >= 70  # Mobile > 70
prob += N2R + J2R + A2R                    >= 80  # Richmond > 80

# objective -- costs of each transfer summed
prob += 30*N2B + 40*N2R + 50*J2M + 45*J2A + 50*J2R + 50*B2C + 35*C2A + 40*A2C + 35*A2M + 30*A2R + 25*M2A 

In [92]:
prob.setSolver(plp.get_solver('PULP_CBC_CMD'))


In [93]:
prob.solve()

1

In [94]:
# look through the values of each variable and the output of the objective function
# for var in prob.variablesDict
print(f"Total cost: {prob.objective.value()}")

for var in prob.variables():
    print(f"{var.name}:\t\t\t{var.value()}")

Total cost: 22350.0
Atlanta_to_Columbus:			40.0
Atlanta_to_Mobile:			0.0
Atlanta_to_Richmond:			0.0
Boston_to_Columbus:			20.0
Columbus_to_Atlanta:			0.0
Jacksonville_to_Atlanta:			210.0
Jacksonville_to_Mobile:			70.0
Jacksonville_to_Richmond:			0.0
Mobile_to_Atlanta:			0.0
Newark_to_Boston:			120.0
Newark_to_Richmond:			80.0


In [95]:
from lpsolve55 import *
from lp_maker import *


In [129]:

# lpsolve so that we can get the sensitivity analysis
# Xj order :
# N2B N2R J2R J2A J2M B2C C2A A2C A2M A2R M2A 

   #     NWK    JAC  BOS  CBS  ATL  MOB  RCH
rhs =  [-200, -300, 100,  60, 170,  70,  80] # rhs of constraint function 
comp = [   1,    1,   1,   1,   1,   1,   1] # -1: lt; 0: eq; 1: gt

# 30*N2B + 40*N2R + 50*J2R+ 45*J2A + 50*J2M + 50*B2C + 35*C2A + 40*A2C + 35*A2M + 30*A2R + 25*M2A 
obj = [30, 40, 50, 45, 50, 50, 35, 40, 35, 30, 25]

      # N2B N2R J2R J2A J2M B2C C2A A2C A2M A2R M2A 
Aij = [[-1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # Newark -- only have 200 cars_to_give. Swapped it to gte to make it easier for the solver
       [ 0,  0, -1, -1, -1,  0,  0,  0,  0,  0,  0],  # Jacksonville -- only have 300 cars_to_give. Swapped it to gte to make it easier for the solver
       [ 1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0],  # Boston -- need_to_have at least 100 cars
       [ 0,  0,  0,  0,  0,  1, -1,  1,  0,  0,  0],  # Columbus -- need_to_have at least 60 cars
       [ 0,  0,  0,  1,  0,  0,  1, -1, -1, -1,  1],  # Atlanta > 170 
       [ 0,  0,  0,  0,  1,  0,  0,  0,  1,  0, -1],  # Mobile > 70
       [ 0,  1,  1,  0,  0,  0,  0,  0,  0,  1,  0]]  # Richmond > 80



lp_handle = lp_maker(obj, Aij, rhs, comp, setminim=1)


In [154]:

trip_names = [N2B, N2R, J2R, J2A, J2M, B2C, C2A, A2C, A2M, A2R, M2A]
cities = ['NWK', 'JAC', 'BOS', 'CBS', 'ATL', 'MOB', 'RCH']
dual_indices = cities+trip_names


In [171]:
lpstat = lpsolve('solve', lp_handle)
print(f"Any failures: {lpstat}")
print(f"Objective Function value: {lpsolve('get_objective', lp_handle)}\n") 
var_sols = lpsolve('get_variables', lp_handle)
for ii in range(len(trip_names)):
    print(f"{trip_names[ii]}: {var_sols[0][ii]:.02f}")

Any failures: 0
Objective Function value: 22350.0

Newark_to_Boston: 120.00
Newark_to_Richmond: 80.00
Jacksonville_to_Richmond: 0.00
Jacksonville_to_Atlanta: 210.00
Jacksonville_to_Mobile: 70.00
Boston_to_Columbus: 20.00
Columbus_to_Atlanta: 0.00
Atlanta_to_Columbus: 40.00
Atlanta_to_Mobile: 0.00
Atlanta_to_Richmond: 0.00
Mobile_to_Atlanta: 0.00


In [166]:
[frm, till, fromval, tilval, ret] = lpsolve('get_sensitivity_obj',lp_handle)
[duals, dualsfrom, dualstill, _] = lpsolve('get_sensitivity_rhs', lp_handle)



sens_pd = pd.DataFrame(np.array([frm, till, fromval]).round().T, index=trip_names, columns=['Lower', 'Upper', 'FromVal'])
dual_pd = pd.DataFrame(np.array([duals, dualsfrom, dualstill]).round().T, index=dual_indices, columns= ['Reduced Cost','Lower Limit','Upper Limit'])

In [167]:
sens_pd.replace(to_replace={-1e+30:-np.inf, 1e30:np.inf})

,Lower,Upper,FromVal
Newark_to_Boston,25.0,35.0,-inf
Newark_to_Richmond,-5.0,45.0,-inf
Jacksonville_to_Richmond,45.0,inf,40.0
Jacksonville_to_Atlanta,40.0,50.0,-inf
Jacksonville_to_Mobile,20.0,80.0,-inf
Boston_to_Columbus,45.0,55.0,-inf
Columbus_to_Atlanta,-40.0,inf,-inf
Atlanta_to_Columbus,35.0,45.0,-inf
Atlanta_to_Mobile,5.0,inf,70.0
Atlanta_to_Richmond,-0.0,inf,40.0


In [168]:
dual_pd.replace(to_replace={-1e+30:-np.inf, 1e30:np.inf})

,Reduced Cost,Lower Limit,Upper Limit
NWK,5.0,-240.0,-180.0
JAC,0.0,-inf,inf
BOS,35.0,60.0,120.0
CBS,85.0,20.0,80.0
ATL,45.0,-40.0,190.0
MOB,50.0,0.0,90.0
RCH,45.0,40.0,100.0
Newark_to_Boston,0.0,-inf,inf
Newark_to_Richmond,0.0,-inf,inf
Jacksonville_to_Richmond,5.0,-20.0,40.0
